### 卷积层基本结构

&emsp;&emsp;先设定卷积层结构如下:

1. 输入特征映射组:$X\subseteq \mathbb{R}^{C \times H \times W} $为三维张量,其中每个切片(通道)矩阵:$X^c \subseteq \mathbb{R}^{H \times W} (1 \leq c \leq C)$为一个特征映射
2. 滤波器:$W \subseteq \mathbb{R}^{FN \times C \times FH \times FW}$为四维张量,其中第$fn$个滤波器的第$c(1 \leq c \leq C)$个通道为:$W^{fn \times c}(1 \leq fn \leq FN,1 \leq c \leq C)$
3. 输出特征映射组:$Y \subseteq \mathbb{R}^{FN \times OH \times OW} $为三维张量,其中激活函数$f$关于第$fn(1 \leq fn \leq FN)$个特征图(也叫特征映射净输入)$Z^{fn}$的输出特征映射为:$ Y^{fn} \subseteq \mathbb{R}^{H \times W}$

易知

\begin{align}
Z^{fn} &=W^{fn} \otimes X + b^{fn}=\sum^{C}_{c=1} W^{fn, c} \otimes X^{c}+b^{fn}   \tag{1} \\
Y^{fn} &= f(Z^{fn})
\end{align}

其中$W^{fn} \subseteq \mathbb{R}^{C \times FH \times FW}$为三维卷积核,$f(*)$为非线性激活函数.


### 池化层反向传播

&emsp;&emsp;当$l+1$层为池化层时,易知此时没有权值参数需要训练,故只需将误差$\delta^{l+1}$反向传播至上一层.

\begin{align}
\boldsymbol{\delta}^{l, fn} & \buildrel \Delta \over = \frac{\partial \mathcal{L}}{\partial Z^{l,fn}} \\
&=    \frac{\partial X^{l,fn}}{\partial Z^{l,fn}} \frac{\partial Z^{l+1,fn}}{\partial X^{l,fn}} \frac{\partial \mathcal{L}}{\partial Z^{l+1,fn}} \\
&=    \frac{\partial f(Z^{l,fn})}{\partial Z^{l,fn}} \frac{\partial Z^{l+1,fn}}{\partial X^{l,fn}} \frac{\partial \mathcal{L}}{\partial Z^{l+1,fn}} \\
&= f'_l(Z^{l,fn}) \odot \mathrm{up}(\boldsymbol{\delta}^{l+1, fn}) 
\end{align}

其中$\odot$表示矩阵对应元素相乘,$f_l(*)$为第$l$层的激活函数(若$l$层无激活函数,则$f'_l(Z^{l,fn})=I$).
$\mathrm{up}$为上采样函数(若池化层下一次为全连接层,则先将$\boldsymbol{\delta}^{l+1}$恢复池化层输出形状,书写顺序为FN,height,widht,其中FN为卷积层滤波器个数),再进行上采样恢复成池化层输入的形状).与池化层中使用的下采样操作刚好相反.如果下采样是Max池化,误差项$\boldsymbol{\delta}^{l+1, fn} $中每个值会直接传递到上一层对应区域中的最大值所对应的神经元,该区域中其他神经元的误差项都设为0.如果下采样是Average池化,误差项$\boldsymbol{\delta}^{l+1, fn }$中每个值会被平均分配到上一层对应区域中的所有神经元上.


### 卷积层反向传播

&emsp;&emsp;当$l+1$层是卷积层时,有

\begin{align}
Z^{l+1, fn} &=W^{l+1,fn} \otimes X + b^{fn}=\sum^{C}_{c=1} W^{l+1,fn,c} \otimes X^{l, c}+ \mathbf{b}^{l+1, fn} \\
X^{l+1,fn} &= f(Z^{l+1,fn})
\end{align}

其中,$Z^{l+1,fn}$表示卷积层输入与第$fn$个滤波器进行卷积的输出,$b^{l+1, fn}$为第$fn$个滤波器的偏置.

***
#### 求导预备知识

&emsp;&emsp;假设$Y = W \otimes X$,其中$X \subseteq \mathbb{R}^{M \times N}$,$W \subseteq \mathbb{R}^{U \times V}$,$Y \subseteq \mathbb{R}^{(M-U+1) \times (N-V+1)}$,函数$f(Y) $为一个标量函数,则

\begin{aligned}
\frac{\partial f(\boldsymbol{Y})}{\partial w_{u v}} &=\sum_{i=1}^{M-U+1} \sum_{j=1}^{N-V+1} \frac{\partial y_{i j}}{\partial w_{u v}} \frac{\partial f(\boldsymbol{Y})}{\partial y_{i j}} \\
&=\sum_{i=1}^{M-U+1} \sum_{j=1}^{N-V+1} x_{i+u-1, j+v-1} \frac{\partial f(\boldsymbol{Y})}{\partial y_{i j}} \quad \quad \text{注:} y_{i j}=\sum_{u, v} w_{u v} x_{i+u-1, j+v-1} \\
&=\sum_{i=1}^{M-U+1} \sum_{j=1}^{N-V+1} \frac{\partial f(\boldsymbol{Y})}{\partial y_{i j}} x_{u+i-1, v+j-1}
\end{aligned}

由互相关卷积的定义可以看出$f(Z)$关于$W$的偏导数为$X$和$\frac{\partial f(Y)}{\partial Y}$的卷积

$$ \frac{\partial f(Y)}{\partial W} = \frac{\partial f(Y)}{\partial Y} \otimes X \tag{2} $$

同理可以得到

\begin{aligned}
\frac{\partial f(Y)}{\partial x_{s t}} &=\sum_{i=1}^{M-U+1} \sum_{j=1}^{N-V+1} \frac{\partial y_{i j}}{\partial x_{s t}} \frac{\partial f(Y)}{\partial y_{i j}} \\
&=\sum_{i=1}^{M-U+1} \sum_{j=1}^{N-V+1} w_{s-i+1, t-j+1} \frac{\partial f(Y)}{\partial y_{i j}} \\
\end{aligned}

易知,当$s-i+1<1$或$s-i+1>U$或$t-j+1<1$或$t-j+1>1$时,$w_{s-i+1, t-j+1}=0$.由翻转卷积公式可得

\begin{align}
\frac{\partial f(Y)} { \partial X} &= \frac{\partial f(Y)}{\partial Y} * \hat{W} \\
		&=\mathrm{rot}180(\frac{\partial f(Y)}{\partial Y})  \otimes \hat{W} \quad \quad \text{注:转换为自相关卷积}  \\
		&= \mathrm{rot}180(\frac{\partial f(Y)}{\partial Y})  \hat{\otimes} W  \\
        &=  \mathrm{rot}180(W) \hat{\otimes} \frac{\partial f(Y)}{\partial Y}  \tag{3}
\end{align}

其中$\hat{W}$是对$W$进行了$P=(M-U,N-V)$的零填充,$\hat{\otimes}$表示宽卷积运算
***

&emsp;&emsp;由公式(2)可得:

\begin{align}
\frac{\partial \mathcal{L}}{W^{l+1,fn,c}} &= \frac{\partial \mathcal{L}}{\partial Z^{l+1,fn}} \otimes X^{l,c} \quad \quad \text{注:这里$\mathcal{L}$是$Z^{l+1,fn}$的函数} \\
&= \boldsymbol{\delta}^{l+1, fn}  \otimes X^{l,c} \quad \quad \text{注:也可去掉$c$进行三维(即方块)卷积运算,易推}
\end{align}

其中$\boldsymbol{\delta}^{l+1, fn} =  \frac{\partial \mathcal{L}}{\partial Z^{l+1,fn}}$为损失函数关于第$l+1$层第$fn$个特征图$Z^{l+1,fn}$的导数.同理可得,

$$ \frac{\partial \mathcal{L}}{\mathbf{b}^{l+1,fn}} = \sum_{i, j}\left[\boldsymbol{\delta}^{(l+1, fn)}\right]_{i, j} $$

根据公式(3),可得:

\begin{aligned}
\boldsymbol{\delta}^{(l, c)} & \triangleq \frac{\partial \mathcal{L}}{\partial Z^{(l, c)}} \\
&=\frac{\partial X^{(l, c)}}{\partial Z^{(l, c)}} \frac{\partial \mathcal{L}}{\partial X^{(l, c)}} \\
&=f_{l}^{\prime}\left(Z^{(l, c)}\right) \odot \sum_{fn=1}^{FN}\left(\mathrm{rot} 180\left(W^{(l+1, fn, c)}\right) \hat{\otimes} \frac{\partial \mathcal{L}}{\partial Z^{(l+1, fn)}}\right) \quad \quad  \text{注:多个滤波器之和} \\
&=f_{l}^{\prime}\left(Z^{(l, fn)}\right) \odot \sum\left(\mathrm{rot} 180\left(W^{(l+1, fn, c)}\right) \hat{\otimes} \boldsymbol{\delta}^{(l+1, fn)}\right)
\end{aligned}

其中$f_l(*)$为第$l$层使用的激活函数(若$l$层不含激活函数,则$f_{l}^{\prime}\left(Z^{(l, fn)} \right) =I$).


### 网络层误差反向传播分析

&emsp;&emsp;假设网络结构为(这里简化为输入数据只有一个通道,多通道同理可得):

??????===>卷积层输入层 ===> 卷积层+激活函数 ===> 池化层 ===> 全连接层+激活函数 ===> Softmax and cross entropy

则误差的反向传播为:

1. Softmax and cross entropy输入关于损失$\mathcal{L}$的误差

    $\boldsymbol{\delta}^{last} =  \operatorname{Softmax} (\mathbf{x}) - \boldsymbol{1}[\text { class }]$

   其中$\mathbf{x}$为Softmax and cross entropy层的输入

2. 全连接层+激活函数输入关于损失$\mathcal{L}$的误差

   $\boldsymbol{\delta}^{full-mesh} =  A^{full-mesh} (W^{full-mesh})^T \boldsymbol{\delta}^{last} $

   其中$A^{full-mesh}=\begin{bmatrix}
   \frac{ \partial f(x_{1})}{\partial x_1} & 0 & \cdots & 0  \\
   0 & \frac{ \partial f(x_{2})}{\partial x_2} & \cdots & 0 \\
   \vdots& \vdots & \ddots & \vdots  \\
   0 & 0 & \cdots & \frac{ \partial f(x_{N})}{\partial x_{N}} \\
   \end{bmatrix} $,其中$x_1,x_2,..\cdots,x_N$为全连接层+激活函数的输入,$f(*)$为该层的激活函数,$W^{full-mesh}$为全连接层权重矩阵.

3. 池化层输入关于损失$\mathcal{L}$的误差

    $$ \boldsymbol{\delta}^{pool} = \mathrm{up}(\boldsymbol{\delta}^{full-mesh}) $$

    其中$\mathrm{up}$为上采样函数(若为Max池化,则需要保存前向传播各个方形数据快max-value的位置索引;若为Average池化,则误差项$\boldsymbol{\delta}^{full-mesh }$中每个值会被平均分配到上一层对应方形数据快中的所有神经元上)

4. 卷积运算输出激活函数输入关于损失$\mathcal{L}$的误差

   $$ \boldsymbol{\delta}^{convo-out} = f_c'(X) \boldsymbol{\delta}^{pool} $$

   其中,$f_c$(*)为卷积层激活函数,$X$为卷积层经过卷积运算的输出

5. 卷积层输入关于损失$\mathcal{L}$的误差

   $$ \boldsymbol{\delta}^{convo-input} = \sum_{fn}^{FN} \left(\mathrm{rot} 180\left(W^{ fn}\right) \hat{\otimes} \boldsymbol{\delta}^{convo-out} \right) $$

   其中$W^{fn}$为第$fn$个滤波器,$\hat{\otimes}$为宽卷积运算.